In [11]:
!pip install pygad numpy sklearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [12]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pygad
import warnings

# Игнорируем предупреждения
warnings.filterwarnings('ignore')

# Загрузка данных Iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
def fitness_function(ga_instance, solution, solution_idx):
    model_type = ga_instance.model_type
    
    if model_type == "logistic":
        # (Previous logistic regression code remains unchanged)
        ...
        
    elif model_type == "tree":
        # Convert parameters to proper types
        max_depth = int(solution[0]) if solution[0] > 0 else None
        
        # Handle min_samples_split and min_samples_leaf
        if solution[1] < 1:  # If less than 1, treat as fraction
            min_samples_split = max(0.01, solution[1])  # Ensure minimum 0.01
        else:
            min_samples_split = int(solution[1])  # Integer ≥1
            
        if solution[2] < 1:  # If less than 1, treat as fraction
            min_samples_leaf = max(0.01, solution[2])  # Ensure minimum 0.01
        else:
            min_samples_leaf = int(solution[2])  # Integer ≥1
            
        criterion = ['gini', 'entropy'][int(solution[3])]
        
        model = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            criterion=criterion,
            random_state=42
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Updated gene space for decision tree to ensure integer values
gene_space_tree = [
    {'low': 0, 'high': 20},     # max_depth (will be converted to int)
    {'low': 2, 'high': 20},     # min_samples_split (will be converted to int)
    {'low': 1, 'high': 20},      # min_samples_leaf (will be converted to int)
    {'low': 0, 'high': 1}       # criterion index
]

# After GA completes for decision tree:
solution, solution_fitness, solution_idx = ga_tree.best_solution()

# Convert parameters properly
best_max_depth = int(solution[0]) if solution[0] > 0 else None

# Handle min_samples_split
if solution[1] < 1:
    best_min_samples_split = max(0.01, solution[1])  # Float fraction
else:
    best_min_samples_split = int(solution[1])  # Integer

# Handle min_samples_leaf
if solution[2] < 1:
    best_min_samples_leaf = max(0.01, solution[2])  # Float fraction
else:
    best_min_samples_leaf = int(solution[2])  # Integer

best_criterion = ['gini', 'entropy'][int(solution[3])]

# Create final model
best_tree = DecisionTreeClassifier(
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    min_samples_leaf=best_min_samples_leaf,
    criterion=best_criterion,
    random_state=42
)

In [14]:
# Параметры генетического алгоритма для логистической регрессии
num_generations = 50
num_parents_mating = 4
sol_per_pop = 8
num_genes = 3

# Границы параметров:
# - C: (0.01, 10)
# - penalty: (0, 3) -> индекс выбора
# - solver: (0, 4) -> индекс выбора
gene_space = [
    {'low': 0.01, 'high': 10},  # C
    {'low': 0, 'high': 3},       # penalty index
    {'low': 0, 'high': 4}        # solver index
]

# Создание экземпляра GA
ga_logistic = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_function,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    gene_space=gene_space,
    save_solutions=True
)

# Установка типа модели
ga_logistic.model_type = "logistic"

# Запуск GA
ga_logistic.run()

# Лучшее решение
solution, solution_fitness, solution_idx = ga_logistic.best_solution()
print(f"Лучшие параметры для логистической регрессии: {solution}, Точность: {solution_fitness}")

# Сохранение лучшей модели логистической регрессии
best_C = solution[0]
best_penalty = ['l1', 'l2', 'elasticnet', 'none'][int(solution[1])]
best_solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'][int(solution[2])]

best_logistic = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    solver=best_solver,
    max_iter=1000,
    random_state=42
)
best_logistic.fit(X_train_scaled, y_train)

# Сохранение модели
import joblib
joblib.dump(best_logistic, 'best_logistic_regression_iris.pkl')

Лучшие параметры для логистической регрессии: [4.32479604 1.12623721 3.56031551], Точность: 1.0


['best_logistic_regression_iris.pkl']

In [17]:
# Параметры генетического алгоритма для дерева решений
num_generations = 50
num_parents_mating = 4
sol_per_pop = 8
num_genes = 4

# Границы параметров:
# - max_depth: (0, 20) -> 0 означает None
# - min_samples_split: (2, 20)
# - min_samples_leaf: (1, 20)
# - criterion: (0, 1) -> индекс выбора
gene_space = [
    {'low': 0, 'high': 20},     # max_depth
    {'low': 2, 'high': 20},     # min_samples_split
    {'low': 1, 'high': 20},      # min_samples_leaf
    {'low': 0, 'high': 1}       # criterion index
]

# Создание экземпляра GA
ga_tree = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_function,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    gene_space=gene_space,
    save_solutions=True
)

# Установка типа модели
ga_tree.model_type = "tree"

# Запуск GA
ga_tree.run()

# Лучшее решение
solution, solution_fitness, solution_idx = ga_tree.best_solution()
print(f"Лучшие параметры для дерева решений: {solution}, Точность: {solution_fitness}")

# Сохранение лучшей модели дерева решений
best_max_depth = int(solution[0]) if solution[0] > 0 else None
best_min_samples_split = solution[1]
best_min_samples_leaf = solution[2]
best_criterion = ['gini', 'entropy'][int(solution[3])]

best_tree = DecisionTreeClassifier(
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    min_samples_leaf=best_min_samples_leaf,
    criterion=best_criterion,
    random_state=42
)
best_tree.fit(X_train, y_train)

# Сохранение модели
joblib.dump(best_tree, 'best_decision_tree_iris.pkl')

Лучшие параметры для дерева решений: [10.85655684 13.65063194  7.22971207  0.75807995], Точность: 1.0


InvalidParameterError: The 'min_samples_leaf' parameter of DecisionTreeClassifier must be an int in the range [1, inf) or a float in the range (0.0, 1.0). Got 7.229712065053356 instead.